In [ ]:
import pymssql
import numpy as np
import pandas as pd
import datetime as dt
from collections import Counter

In [ ]:
database = "arctic_analysts_capstone"
user = "arctic_analysts"
password  = "ThisPassw0rd!"
server = "gen10-data-fundamentals-22-02-sql-server.database.windows.net"

def get_data():
    conn = pymssql.connect(server, user, password, database)
    cursor = conn.cursor()
    
    query = f'SELECT * FROM median_income'

    income_df = pd.read_sql(query, conn)
    print(f'\nBefore Data Management: {income_df.shape[0]}')

    return income_df

In [ ]:
income_df = get_data()

In [ ]:
coconino_county = income_df[(income_df.county == 'Coconino County, Arizona')]

In [ ]:
coconino_county

In [ ]:
overall_coconino = coconino_county[(coconino_county.AgeGroup == 'overall')].copy()
overall_coconino.sort_values(by = 'year')

In [ ]:
from sklearn.linear_model import ElasticNetCV

In [ ]:
def get_step(df):
    df.sort_values(by = 'year', inplace = True)
    for i in range(1, 7):
        col_name = 'Step_{}'.format(i)
        df[col_name] = df['year'].shift(periods=-1 * i)
    df = df.dropna().reset_index(drop = True)
    return df
stepped_df = get_step(overall_coconino[['year', 'MedianIncome']].copy())
stepped_df

In [ ]:
import plotly.express as px

In [ ]:
print(sum([.68, .155, .05, .037, .035, .023, .01, .01]))

In [ ]:
fig = px.line(overall_coconino, x = 'year', y='MedianIncome')

In [ ]:
fig.show()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from matplotlib import rcParams
from cycler import cycler

In [ ]:
overall_coconino.set_index('year', inplace = True)
overall_coconino.index.freq = 'YS'

In [ ]:
overall_coconino = overall_coconino[['MedianIncome']]

In [ ]:
overall_coconino.reset_index(inplace = True)

In [ ]:
records = len(overall_coconino)

df_train = overall_coconino[:-int(records *.25)]
df_test = overall_coconino[-int(records*.25):]

In [ ]:
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing
import time

In [ ]:
for periods in range(2, 15): 

    model_mul_additive = ExponentialSmoothing(df_train['MedianIncome'], trend='mul', seasonal='add', seasonal_periods = periods)
    results_mul_add = model_mul_additive.fit()
    predictions_mul_add = results_mul_add.forecast(steps=3)
    rmse_mul_add = mean_squared_error(df_test['MedianIncome'], predictions_mul_add, squared=False)

    model_mul_mult = ExponentialSmoothing(df_train['MedianIncome'], trend='mul', seasonal='mul', seasonal_periods = periods)
    results_mul_mult = model_mul_mult.fit()
    predictions_mul_mult = results_mul_mult.forecast(steps=3)
    rmse_mul_mult = mean_squared_error(df_test['MedianIncome'], predictions_mul_mult, squared=False)

    fig1 = plt.figure(figsize = (17,5))

    ax1 = plt.plot(overall_coconino['MedianIncome'], color = 'black')
    ax1 = plt.plot(predictions_mul_add, color ='orange')
    ax1 = plt.plot(df_test['MedianIncome'], color = 'green', linewidth = 3)
    ax1 = plt.plot(predictions_mul_mult, color='red')
    plt.show()
    
    

In [ ]:
df_test